In [1]:
import gzip
import math
import sklearn
import random
import numpy as np
import pandas as pd
import sklearn
from pandas import DataFrame
from sklearn import linear_model
from collections import defaultdict
from random import choice
import matplotlib.pyplot as plt

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        true=True
        false=False
        yield eval(l)

def readJSON(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [3]:
dataset = pd.read_csv("winequality-red.csv")

In [25]:
def feature1(datum):
    fea=list()
    for i in range(len(datum)):
        f_acid=list(datum['fixed acidity'])[i]/(datum['fixed acidity'].max())
        v_acid=list(datum['volatile acidity'])[i]/(datum['volatile acidity'].max())
        c_acid=list(datum['citric acid'])[i]/(np.array(datum['citric acid']).max())
        r_sugar=list(datum['residual sugar'])[i]/(np.array(datum['residual sugar']).max())
        chlor=list(datum['chlorides'])[i]/(np.array(datum['chlorides']).max())
        f_sulfur=list(datum['free sulfur dioxide'])[i]/(np.array(datum['free sulfur dioxide']).max())
        t_sulfur=list(datum['total sulfur dioxide'])[i]/(np.array(datum['total sulfur dioxide']).max())
        density=list(datum['density'])[i]/(np.array(datum['density']).max())
        ph=list(datum['pH'])[i]/(np.array(datum['pH']).max())
        sulphates=list(datum['sulphates'])[i]/(np.array(datum['sulphates']).max())
        alch=list(datum['alcohol'])[i]/(np.array(datum['alcohol']).max())
        a=[f_acid]+[v_acid]+[c_acid]+[r_sugar]+[chlor]+[f_sulfur]+[t_sulfur]+[density]+[ph]+[sulphates]+[alch]
        fea.append(a)
    return fea
#normalize all feature

In [14]:
X=feature1(dataset)
y=dataset['quality']

In [40]:
index=2*len(X)//3
X_train=X[:index]
X_test=X[index:]
y_train=y[:index]
y_test=y[index:]

In [21]:
#calculate covariance

In [16]:
X_def=np.array(X_train)
X_def=np.mat(X_train).T

In [17]:
Xmean=np.mean(X_def, axis=1)
X_def2=X_def-Xmean

In [18]:
covi=np.cov(X_def2,rowvar=True)
eigenValue, eigenVec=np.linalg.eig(covi)

In [19]:
np.mean(covi,axis=1)

array([ 1.96629023e-03, -3.71620516e-04,  4.48901542e-03,  1.15644654e-03,
        1.03363666e-03,  2.55788499e-03,  2.10782880e-03,  2.63665543e-05,
       -5.29862123e-04,  1.52679918e-03,  4.38946386e-04])

In [20]:
#chi-square test

In [22]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
model1 = SelectKBest(chi2, k=5)
model1.fit_transform(X_train,y_train)

array([[0.44303797, 0.        , 0.15277778, 0.11764706, 0.63087248],
       [0.55696203, 0.        , 0.34722222, 0.23183391, 0.65771812],
       [0.48101266, 0.04      , 0.20833333, 0.18685121, 0.65771812],
       ...,
       [0.1835443 , 0.65      , 0.08333333, 0.05882353, 0.79194631],
       [0.46835443, 0.09      , 0.06944444, 0.03460208, 0.79194631],
       [0.38607595, 0.18      , 0.08333333, 0.06920415, 0.68456376]])

In [23]:
model1.scores_

array([8.50108874e-01, 5.42409239e+00, 7.77535264e+00, 3.60636196e-01,
       6.35979363e-01, 1.66514630e+00, 8.95288964e+00, 1.31118037e-04,
       1.31322977e-02, 1.18014537e+00, 2.27387537e+00])

In [24]:
#linear regression

In [26]:
theta,residuals,rank,s = np.linalg.lstsq(X_train, y_train)

<ipython-input-26-929a17f25f37>:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  theta,residuals,rank,s = np.linalg.lstsq(X_train, y_train)


In [27]:
theta

array([ 0.0083251 , -1.59819856, -0.22507577,  0.27441969, -1.00113619,
        0.21680442, -1.31827605,  4.21887481, -1.69847189,  1.39235898,
        4.68206521])

In [28]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [37]:
model = sklearn.linear_model.LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [46]:
yPred = model.predict(X_test)

In [48]:
a=model.score(X_test,y_test)
b=MSE(yPred,y_test)
print('MSE is ', b, 'R^2 is ',a)

MSE is  0.4497687436793937 R^2 is  0.309327823658728


In [49]:
def feature2(datum,j):
    fea=list()
    for i in range(len(datum)):
        f_acid=list(datum['fixed acidity'])[i]/(datum['fixed acidity'].max())
        v_acid=list(datum['volatile acidity'])[i]/(datum['volatile acidity'].max())
        c_acid=list(datum['citric acid'])[i]/(np.array(datum['citric acid']).max())
        r_sugar=list(datum['residual sugar'])[i]/(np.array(datum['residual sugar']).max())
        chlor=list(datum['chlorides'])[i]/(np.array(datum['chlorides']).max())
        f_sulfur=list(datum['free sulfur dioxide'])[i]/(np.array(datum['free sulfur dioxide']).max())
        t_sulfur=list(datum['total sulfur dioxide'])[i]/(np.array(datum['total sulfur dioxide']).max())
        density=list(datum['density'])[i]/(np.array(datum['density']).max())
        ph=list(datum['pH'])[i]/(np.array(datum['pH']).max())
        sulphates=list(datum['sulphates'])[i]/(np.array(datum['sulphates']).max())
        alch=list(datum['alcohol'])[i]/(np.array(datum['alcohol']).max())
        a=[1]+[f_acid]+[v_acid]+[c_acid]+[r_sugar]+[chlor]+[f_sulfur]+[t_sulfur]+[density]+[ph]+[sulphates]+[alch]
        a.pop(j) 
        fea.append(a)
    return fea

In [52]:
y_train=y[:index]
y_test=y[index:]
mse_min=0.5
j=1
while j <12:
    X=feature2(dataset,j)
    X_train=X[:index]
    X_test=X[index:]
    model = sklearn.linear_model.LinearRegression()
    model.fit(X_train,y_train)
    yPred = model.predict(X_test)
    p=MSE(yPred, y_test)-b
    print(j,' MSE is ', p)
    j+=1

1  MSE is  -5.9175796987720464e-05
2  MSE is  0.03174141969802796
3  MSE is  -5.489917844514469e-05
4  MSE is  -0.0019288777140277213
5  MSE is  0.007705739866039618
6  MSE is  0.00159699601817892
7  MSE is  -0.003396528902696372
8  MSE is  -0.001660091842064837
9  MSE is  0.0019488338065184463
10  MSE is  0.023232617637566566
11  MSE is  0.023019413953294876


In [62]:
def feature3(datum):
    fea=list()
    for i in range(len(datum)):
        f_acid=list(datum['fixed acidity'])[i]/(datum['fixed acidity'].max())
        v_acid=list(datum['volatile acidity'])[i]/(datum['volatile acidity'].max())
        c_acid=list(datum['citric acid'])[i]/(np.array(datum['citric acid']).max())
        r_sugar=list(datum['residual sugar'])[i]/(np.array(datum['residual sugar']).max())
        chlor=list(datum['chlorides'])[i]/(np.array(datum['chlorides']).max())
        f_sulfur=list(datum['free sulfur dioxide'])[i]/(np.array(datum['free sulfur dioxide']).max())
        t_sulfur=list(datum['total sulfur dioxide'])[i]/(np.array(datum['total sulfur dioxide']).max())
        density=list(datum['density'])[i]/(np.array(datum['density']).max())
        ph=list(datum['pH'])[i]/(np.array(datum['pH']).max())
        sulphates=list(datum['sulphates'])[i]/(np.array(datum['sulphates']).max())
        alch=list(datum['alcohol'])[i]/(np.array(datum['alcohol']).max())
        a=[1]+[f_acid]+[v_acid]+[r_sugar]+[chlor]+[ph]+[sulphates]+[alch]
        fea.append(a)
    return fea

In [63]:
#Based on above method, we finally decided to remove 4 features: citric acid,free sulfur dioxide,
#and total sulfur dioxide,density

X1=feature3(dataset)
X1_train=X1[:index]
X1_test=X1[index:]
model = sklearn.linear_model.LinearRegression()
model.fit(X1_train,y_train)
yPred = model.predict(X1_test)
MSE(yPred, y_test)

0.4262891888004292